# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 7 2022 2:09PM,241347,21,sample-402140895,"NBTY Global, Inc.",Released
1,Jul 7 2022 2:06PM,241346,19,DEX0007143,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jul 7 2022 2:06PM,241346,19,DEX0007144,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jul 7 2022 2:06PM,241346,19,DEX0007145,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,Jul 7 2022 2:01PM,241341,10,0085842375,ISDIN Corporation,Released
5,Jul 7 2022 2:01PM,241341,10,0085842369,ISDIN Corporation,Released
6,Jul 7 2022 2:01PM,241341,10,0085842373,ISDIN Corporation,Released
7,Jul 7 2022 2:01PM,241341,10,0085842370,ISDIN Corporation,Released
8,Jul 7 2022 2:01PM,241341,10,0085842371,ISDIN Corporation,Released
9,Jul 7 2022 2:01PM,241341,10,0085842372,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,241343,Released,1
49,241344,Released,1
50,241345,Released,1
51,241346,Released,3
52,241347,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241343,NaN,1.0
241344,NaN,1.0
241345,NaN,1.0
241346,NaN,3.0
241347,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241228,0.0,1.0
241243,2.0,3.0
241244,0.0,1.0
241245,0.0,1.0
241246,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241228,0,1
241243,2,3
241244,0,1
241245,0,1
241246,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241228,0,1
1,241243,2,3
2,241244,0,1
3,241245,0,1
4,241246,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241228,,1
1,241243,2,3
2,241244,,1
3,241245,,1
4,241246,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 7 2022 2:09PM,241347,21,"NBTY Global, Inc."
1,Jul 7 2022 2:06PM,241346,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
4,Jul 7 2022 2:01PM,241341,10,ISDIN Corporation
53,Jul 7 2022 1:56PM,241345,21,"NBTY Global, Inc."
54,Jul 7 2022 1:51PM,241344,21,"NBTY Global, Inc."
55,Jul 7 2022 1:47PM,241343,21,"NBTY Global, Inc."
56,Jul 7 2022 1:34PM,241337,20,Alumier Labs Inc.
74,Jul 7 2022 1:29PM,241338,19,"GCH Granules USA, Inc."
75,Jul 7 2022 1:25PM,241336,10,"Citieffe, Inc."
76,Jul 7 2022 1:05PM,241334,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 7 2022 2:09PM,241347,21,"NBTY Global, Inc.",,1
1,Jul 7 2022 2:06PM,241346,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,3
2,Jul 7 2022 2:01PM,241341,10,ISDIN Corporation,,49
3,Jul 7 2022 1:56PM,241345,21,"NBTY Global, Inc.",,1
4,Jul 7 2022 1:51PM,241344,21,"NBTY Global, Inc.",,1
5,Jul 7 2022 1:47PM,241343,21,"NBTY Global, Inc.",,1
6,Jul 7 2022 1:34PM,241337,20,Alumier Labs Inc.,,18
7,Jul 7 2022 1:29PM,241338,19,"GCH Granules USA, Inc.",,1
8,Jul 7 2022 1:25PM,241336,10,"Citieffe, Inc.",,1
9,Jul 7 2022 1:05PM,241334,10,"Methapharm, Inc.",,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 2:09PM,241347,21,"NBTY Global, Inc.",1,
1,Jul 7 2022 2:06PM,241346,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3,
2,Jul 7 2022 2:01PM,241341,10,ISDIN Corporation,49,
3,Jul 7 2022 1:56PM,241345,21,"NBTY Global, Inc.",1,
4,Jul 7 2022 1:51PM,241344,21,"NBTY Global, Inc.",1,
5,Jul 7 2022 1:47PM,241343,21,"NBTY Global, Inc.",1,
6,Jul 7 2022 1:34PM,241337,20,Alumier Labs Inc.,18,
7,Jul 7 2022 1:29PM,241338,19,"GCH Granules USA, Inc.",1,
8,Jul 7 2022 1:25PM,241336,10,"Citieffe, Inc.",1,
9,Jul 7 2022 1:05PM,241334,10,"Methapharm, Inc.",5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 2:09PM,241347,21,"NBTY Global, Inc.",1,
1,Jul 7 2022 2:06PM,241346,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3,
2,Jul 7 2022 2:01PM,241341,10,ISDIN Corporation,49,
3,Jul 7 2022 1:56PM,241345,21,"NBTY Global, Inc.",1,
4,Jul 7 2022 1:51PM,241344,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 2:09PM,241347,21,"NBTY Global, Inc.",1.0,NaN
1,Jul 7 2022 2:06PM,241346,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3.0,NaN
2,Jul 7 2022 2:01PM,241341,10,ISDIN Corporation,49.0,NaN
3,Jul 7 2022 1:56PM,241345,21,"NBTY Global, Inc.",1.0,NaN
4,Jul 7 2022 1:51PM,241344,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 2:09PM,241347,21,"NBTY Global, Inc.",1.0,0.0
1,Jul 7 2022 2:06PM,241346,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3.0,0.0
2,Jul 7 2022 2:01PM,241341,10,ISDIN Corporation,49.0,0.0
3,Jul 7 2022 1:56PM,241345,21,"NBTY Global, Inc.",1.0,0.0
4,Jul 7 2022 1:51PM,241344,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5790832,333.0,9.0
102,965249,4.0,0.0
16,241256,1.0,0.0
19,723942,10.0,0.0
20,723936,82.0,2.0
21,2654186,10.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5790832,333.0,9.0
1,102,965249,4.0,0.0
2,16,241256,1.0,0.0
3,19,723942,10.0,0.0
4,20,723936,82.0,2.0
5,21,2654186,10.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,333.0,9.0
1,102,4.0,0.0
2,16,1.0,0.0
3,19,10.0,0.0
4,20,82.0,2.0
5,21,10.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,333.0
1,102,Released,4.0
2,16,Released,1.0
3,19,Released,10.0
4,20,Released,82.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,16,19,20,21
Status,,,,,,
Executing,9.0,0.0,0.0,0.0,2.0,1.0
Released,333.0,4.0,1.0,10.0,82.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,16,19,20,21
0,Executing,9.0,0.0,0.0,0.0,2.0,1.0
1,Released,333.0,4.0,1.0,10.0,82.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,16,19,20,21
0,Executing,9.0,0.0,0.0,0.0,2.0,1.0
1,Released,333.0,4.0,1.0,10.0,82.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()